In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy
import sys

0.17


In [333]:
# print(os.path.abspath(os.curdir))
# os.chdir(os.path.abspath(os.curdir))

In [334]:
clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

In [336]:
pred_alc= clf_alc_initial.predict(df)
print(pred_alc[:10])

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


[0 1 1 1 1 1 1 1 0 1]


In [337]:
from sklearn.metrics import roc_curve, auc

In [338]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import twokenize.twokenize as tokenizer

from pipelines.helpers import ItemGetter

def make_classifier():
    clf = Pipeline([
        ("getter", ItemGetter("text")),
        ("tfidf", TfidfVectorizer()),
        ("clf", LogisticRegression())])

    clf_params = {
        'clf__C': 200,
        'clf__dual': False,
        'clf__max_iter': 100,
        'clf__multi_class': 'ovr',
        'clf__penalty': 'l2',
        'tfidf__tokenizer':tokenizer.tokenize,
        'tfidf__ngram_range':(1, 3),
        'tfidf__max_features':200000
    }

    clf.set_params(**clf_params)
    return clf

In [339]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651


In [340]:
# print(df.text[:2])
# print(alc_labels[:2])

In [341]:
# split data
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.1, random_state=26)
# print(df.labels[:5])

In [342]:
# df[:5]
# X_train[:5]
# X_test[:5]
# y_train[:5]
# y_test[:5]
# print('training set:', len(X_train))
# print('testing set:', len(X_test))

In [343]:
clf_alc_new = make_classifier()

In [344]:
%%time
clf_alc_new.fit(X_train, y_train)
# pred_alc_new= clf_alc.predict(X_test)
# pred_alc_new= clf_alc.predict(df)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Wall time: 4.25 s


Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=200000, min_df=1,
        ngram_range=(1, 3), norm='l2',...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [345]:
y_alc_new= clf_alc_new.predict(X_test)
# pred_alc_new= clf_alc.predict(df)
print(y_alc_new[:20])
print(y_test[:20])
# print(len(pred_alc_new))
# print(len(y_test))

[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1]
[1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [346]:
# def computeAccuracy(y_predict, y_gt):
# # comps = y_test[:100] != pred_alc_new[:100]
#     comps = y_gt != y_predict
#     # print(comps[:100])
#     print('length of total comps: ', len(comps))
#     diff = list(numpy.where(comps)[0])
#     # print('length of difference: ', len(diff))
#     # print(diff)
#     print('pretrained alcohol classifer:  %d different item in whole dataset'%(len(diff)))
#     acc = (len(comps)-len(diff))/len(comps)*100
#     print('accuracy: %f%%'%(acc))
#     return acc

In [347]:
# from sklearn.metrics import f1_score
# def computeF1Score(y_gt, y_predict):
# #     kwargs = {}
# #     kwargs["average"] = 'micro'
# #     f1 = f1_score(y_true=y_gt, y_pred=y_alc_initial, **kwargs)
#     f1 = f1_score(y_true=y_gt, y_pred=y_predict)
#     return f1

In [348]:
from evaluation.metric import computeAccuracy, computeF1Score

In [349]:
computeAccuracy(y_alc_new, y_test)

length of total comps:  1566
pretrained alcohol classifer:  288 different item in whole dataset
accuracy: 81.609195%


81.60919540229885

In [350]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
auc_score_new = roc_auc_score(y_test,pred_alc_new)
print('alcohol LR AUC: ', auc_score_new)

alcohol LR AUC:  0.7702148657941845


In [351]:
clf_alc_initial

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...one,
          solver='liblinear', tol=0.000655077907893521, verbose=0,
          warm_start=None))])

Predict in whole dataset to see the accuracy of initial loaded model

In [352]:
y_whole_alc_initial = clf_alc_initial.predict(df)
computeAccuracy(y_whole_alc_initial, alc_labels)
auc_score_initial = roc_auc_score(alc_labels,y_whole_alc_initial)
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  15651
pretrained alcohol classifer:  8 different item in whole dataset
accuracy: 99.948885%
alcohol LR AUC:  0.9994771039256063


Training in training set

In [353]:
import numpy as np
# clf_alc_initial.fit(X_train, np.asarray(y_train))
clf_alc_initial.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...one,
          solver='liblinear', tol=0.000655077907893521, verbose=0,
          warm_start=None))])

Predict in testing set

In [354]:
# predict in testing set.
y_alc_initial = clf_alc_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_initial, y_test)
auc_score_initial = roc_auc_score(y_test,y_alc_initial)
f1 = computeF1Score(y_test, y_alc_initial)
print('F1 Score: ', f1)
print('alcohol LR AUC: ', auc_score_initial)

length of total comps:  1566
pretrained alcohol classifer:  261 different item in whole dataset
accuracy: 83.333333%
F1 Score:  0.8718703976435936
alcohol LR AUC:  0.8099516512591146


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [355]:
# test in training set to figure out why the test accuracy is low
y_train_alc_initial = clf_alc_initial.predict(X_train)
computeAccuracy(y_train_alc_initial, y_train)
auc_score_initial = roc_auc_score(y_train,y_train_alc_initial)
f1 = computeF1Score(y_train, y_train_alc_initial)
print('F1 Score: ', f1)
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  14085
pretrained alcohol classifer:  3 different item in whole dataset
accuracy: 99.978701%
F1 Score:  0.9998388396454472
alcohol LR AUC:  0.9997878973332235


Refit whole dataset instead of training-testing split dataset to see, if we can get the same wrong number as in initial model loaded from pickle.

In [356]:
# # refit whole dataset instead of training-testing split to see, if we can get the same wrong number as in initial model loaded from pickle.
# clf_alc_initial.fit(df, alc_labels)
# y_whole_alc_initial = clf_alc_initial.predict(df)
# computeAccuracy(y_whole_alc_initial, alc_labels)
# auc_score_initial = roc_auc_score(alc_labels,y_whole_alc_initial)
# print('alcohol LR AUC: ', auc_score_initial)

Report function from Jason's work. Include confusion matrix. Maybe used in the future

In [357]:
# from classification.reporting import ClassificationReporting
# reporting_new = ClassificationReporting(clf_alc_new, X_train, X_test, y_train, y_test, 2)
# reporting_initial = ClassificationReporting(clf_alc_initial, X_train, X_test, y_train, y_test, 2)

In [358]:
# %%time
# report_new = reporting_new.set_name("Test new classifier").set_level('alc_new').create_report(1)

In [359]:
# report_initial = reporting_initial.set_name("Test initial classifier").set_level('alc_initial').create_report(1)

Trying to convert the LR model to SVM model.

In [360]:
# clf_alc_initial.set_params(clf_alc_initial.named_steps['clf']=LinearSVC())
clf_alc_initial.named_steps['clf']=LinearSVC()

In [361]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import FeatureUnion
# params = {'features': FeatureUnion()}
# clf_alc_SVM = clf_alc_initial.set_params(**params)
# params = {'clf': LinearSVC()}
# clf_alc_SVM = clf_alc_initial.set_params(**params)
# params = {'steps[2]'=LinearSVC()}
# clf_alc_SVM = clf_alc_initial.set_params(**params)
# LinearSVC()
# clf_alc_initial.fit(X_train, y_train, clf__sample_weight=np.array([3,4,2,3]))
#  = LinearSVC())
FeatureUnion(transformer_list='text')

FeatureUnion(n_jobs=1, transformer_list='text', transformer_weights=None)

In [362]:
print(type(clf_alc_initial))
print(type(sub_model))

sub_model = clf_alc_initial.steps
# sub_model[2]
params_SVM = {'clf': LinearSVC()}
sub_model_SVM = sub_model.set_params(**params_SVM)

<class 'sklearn.pipeline.Pipeline'>
<class 'list'>


AttributeError: 'list' object has no attribute 'set_params'

In [ ]:
clf_alc.steps